In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as re
import xmltodict as xml2dict
from collections import OrderedDict

%matplotlib inline

In [34]:
url = 'http://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes'
params = {
    'sigla':'',
    'numero':'',
    'ano':'',
    'datApresentacaoIni':'',
    'datApresentacaoFim':'',
    'parteNomeAutor':'',
    'idTipoAutor':'',
    'siglaPartidoAutor':'',
    'siglaUFAutor':'',
    'generoAutor':'',
    'codEstado':'',
    'codOrgaoEstado':'',
    'emTramitacao':''
}

In [35]:
params['sigla'] = 'PL'
params['ano'] = '2017'
response = re.get(url, params)
print('URL: %s\nStatus: %s' % (response.url, response.status_code))

URL: http://www.camara.leg.br/SitCamaraWS/Proposicoes.asmx/ListarProposicoes?generoAutor=&siglaUFAutor=&ano=2017&numero=&codOrgaoEstado=&emTramitacao=&datApresentacaoIni=&idTipoAutor=&siglaPartidoAutor=&codEstado=&datApresentacaoFim=&parteNomeAutor=&sigla=PL
Status: 200


In [36]:
tree = xml2dict.parse(response.content)


In [37]:
tree.keys()

[u'proposicoes']

In [38]:
proposicoes = tree['proposicoes']
[x for x in proposicoes.keys()]

[u'proposicao']

In [39]:
proposicoes['proposicao'][0].keys()

[u'id',
 u'nome',
 u'tipoProposicao',
 u'numero',
 u'ano',
 u'orgaoNumerador',
 u'datApresentacao',
 u'txtEmenta',
 u'txtExplicacaoEmenta',
 u'regime',
 u'apreciacao',
 u'autor1',
 u'qtdAutores',
 u'ultimoDespacho',
 u'situacao',
 u'indGenero',
 u'qtdOrgaosComEstado']

In [82]:
proposicao = proposicoes['proposicao']

## Nosso resultado tem dicionários aninhados, segue o algoritmo que resolve esse problema

In [83]:
new_dict = {}
chaves_to_delete = []

for prop in proposicao:
    for item in prop:
        if isinstance(prop[item], OrderedDict):
            for x in prop[item].items():
                new_dict[item+'_'+x[0]] = x[1]
            chaves_to_delete.append(item)
            prop.update(new_dict)

In [84]:
for prop in proposicao:
    for chave in prop.keys():
        if chave in chaves_to_delete:
            del prop[chave]

In [103]:
proposicao_df = pd.DataFrame(proposicao)
proposicao_df.head()
print(len(proposicao_df))

2299


In [104]:
shape = proposicao_df.shape
print('%s linhas, %s colunas' % (shape[0], shape[1]))

2299 linhas, 33 colunas


In [105]:
proposicao_df.columns

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_id', u'tipoProposicao_sigla',
       u'tipoProposicao_nome', u'orgaoNumerador_sigla', u'orgaoNumerador_nome',
       u'orgaoNumerador_id', u'regime_txtRegime', u'regime_codRegime',
       u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'autor1_txtSiglaPartido', u'autor1_idecadastro',
       u'autor1_txtNomeAutor', u'autor1_codPartido', u'autor1_txtSiglaUF',
       u'ultimoDespacho_datDespacho', u'ultimoDespacho_txtDespacho',
       u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')

In [115]:
print(proposicao_df.columns)
proposicao_clean = proposicao_df[pd.notnull(proposicao_df['autor1_idecadastro'])]

Index([u'id', u'nome', u'numero', u'ano', u'datApresentacao', u'txtEmenta',
       u'txtExplicacaoEmenta', u'qtdAutores', u'indGenero',
       u'qtdOrgaosComEstado', u'tipoProposicao_id', u'tipoProposicao_sigla',
       u'tipoProposicao_nome', u'orgaoNumerador_sigla', u'orgaoNumerador_nome',
       u'orgaoNumerador_id', u'regime_txtRegime', u'regime_codRegime',
       u'apreciacao_txtApreciacao', u'apreciacao_id',
       u'autor1_txtSiglaPartido', u'autor1_idecadastro',
       u'autor1_txtNomeAutor', u'autor1_codPartido', u'autor1_txtSiglaUF',
       u'ultimoDespacho_datDespacho', u'ultimoDespacho_txtDespacho',
       u'situacao_id', u'situacao_descricao',
       u'situacao_principal_proposicaoPrincipal',
       u'situacao_principal_codProposicaoPrincipal',
       u'situacao_orgao_codOrgaoEstado', u'situacao_orgao_siglaOrgaoEstado'],
      dtype='object')


In [118]:
proposicao_clean.to_csv('../data/proposicoes_PL.csv', sep=',', encoding='utf-8')

### Podemos ver que nosso dataset possui a relação que precisávamos *( deputados <-> proposições )*

In [125]:
p = pd.read_csv('../data/proposicoes_PL.csv')
p[['autor1_idecadastro', 'autor1_txtNomeAutor']].head()

,autor1_idecadastro,autor1_txtNomeAutor
0,178828,Evandro Gussi
1,195866,Vaidon Oliveira
2,74787,Milton Monti
3,141401,Carlos Bezerra
4,74399,Onyx Lorenzoni
